In [17]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [18]:
def load_data(file_path):
    # CSV 파일 읽기
    df = pd.read_csv(file_path)
    
    # 특성과 레이블 분리
    X = df.drop(['people'], axis=1)
    y = df['people']
    
    # 문자열 레이블을 숫자로 변환
    le = LabelEncoder()
    y = le.fit_transform(y)
    
    # 숫자형 데이터만 선택하여 정규화
    numeric_columns = X.select_dtypes(include=[np.number]).columns
    X = X[numeric_columns].values
    
    # 데이터 정규화
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    
    # 3D 텐서로 reshape (samples, timesteps, features)
    X = X.reshape((X.shape[0], X.shape[1], 1))
    
    return X, y, le.classes_

In [19]:
def main():
    # 데이터 로드
    X, y, class_names = load_data("C:/dataset/DSA_features.csv")
    num_classes = len(class_names)
    print("클래스:", class_names)
    print("특성 형태:", X.shape)
    
    # 훈련/검증/테스트 세트 분할
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
    
    # 모델 파라미터 설정
    input_shape = X_train.shape[1:]
    head_size = 256
    num_heads = 4
    ff_dim = 4
    num_transformer_blocks = 4
    mlp_units = [128]
    mlp_dropout = 0.4
    dropout = 0.25
    
    # 모델 생성
    model = build_model(
        input_shape,
        head_size,
        num_heads,
        ff_dim,
        num_transformer_blocks,
        mlp_units,
        dropout,
        mlp_dropout,
        num_classes
    )
    
    # 모델 컴파일
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-4),
        loss="sparse_categorical_crossentropy",
        metrics=["sparse_categorical_accuracy"],
    )
    
    # 모델 요약
    model.summary()
    
    # Early Stopping 콜백
    callbacks = [
        keras.callbacks.EarlyStopping(
            patience=10,
            restore_best_weights=True,
        )
    ]
    
    # 모델 학습
    history = model.fit(
        X_train,
        y_train,
        validation_data=(X_val, y_val),
        epochs=100,
        batch_size=64,
        callbacks=callbacks,
    )
    
    # 모델 평가
    test_loss, test_acc = model.evaluate(X_test, y_test)
    print(f"\nTest accuracy: {test_acc:.4f}")

if __name__ == "__main__":
    main()

클래스: ['p1' 'p2' 'p3' 'p4' 'p5' 'p6' 'p7' 'p8']
특성 형태: (9120, 270, 1)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 270, 1)            │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ multi_head_attention          │ (None, 270, 1)            │           7,169 │ input_layer[0][0],         │
│ (MultiHeadAttention)          │                           │                 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_1 (Dropout)           │ (None, 270, 1)            │               0 │ multi_head_attention[0][0] │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization           │ (None, 270, 1)            │               2 │ dropout_1[0][0]            │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add (Add)                     │ (None, 270, 1)            │               0 │ layer_normalization[0][0], │
│                               │                           │                 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d (Conv1D)               │ (None, 270, 4)            │               8 │ add[0][0]                  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_2 (Dropout)           │ (None, 270, 4)            │               0 │ conv1d[0][0]               │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_1 (Conv1D)             │ (None, 270, 1)            │               5 │ dropout_2[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization_1         │ (None, 270, 1)            │               2 │ conv1d_1[0][0]             │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense (Dense)                 │ (None, 270, 32)           │              64 │ layer_normalization_1[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_1 (Add)                   │ (None, 270, 32)           │               0 │ dense[0][0], add[0][0]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ multi_head_attention_1        │ (None, 270, 32)           │         134,176 │ add_1[0][0], add_1[0][0]   │
│ (MultiHeadAttention)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_4 (Dropout)           │ (None, 270, 32)           │               0 │ multi_head_attention_1[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization_2         │ (None, 270, 32)           │              64 │ dropout_4[0][0]            │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 449,926 (1.72 MB)

 Trainable params: 449,926 (1.72 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
18/92 ━━━━━━━━━━━━━━━━━━━━ 3:08 3s/step - loss: 3.6818 - sparse_categorical_accuracy: 0.1182

KeyboardInterrupt: 